In [9]:
# load data files
# PLEASE MODIFY THE PATH OF DATA!!!!!!!!!!!!!!!!!!!!!!!!!
# ===========================================================================================
# 下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下下

data_subfold = "data_2023_03_01-16_48_29"    # the data from Lukas
# data_subfold = "rosbag2_2023_06_05-16_57_44"
# data_subfold = "rosbag2_2023_06_05-17_22_10"
# data_subfold = "rosbag2_2023_06_05-17_26_41"

# data_subfold = "rosbag2_2023_06_16-16_50_38"


save_merged_data = False

execfile("data_process_from_rosbags.py")
# 上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上上
# ===========================================================================================


# Files
from model import *
from model_kf import *
from model_all import *

# Packages
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

def plot_config(width: int, height: int,title=None,
                x_label: str = None, 
                y_label: str = None):
    p = figure(title=title, background_fill_color="#fafafa", width=width, height=height)
    p.output_backend = "svg"    # save plot in HTML as SVG
    p.title.text_font_size = "15pt"
    p.xaxis.axis_label = x_label
    p.yaxis.axis_label = y_label
    p.yaxis.axis_label_text_font_size = "12pt"
    p.yaxis.major_label_text_font_size = "12pt"
    p.xaxis.major_label_text_font_size = "12pt"
    p.xaxis.axis_label_text_font_size = "12pt"
    return p

'data.hamster2__command.csv' loaded to 'df_command'
'data.hamster2__imu.csv' loaded to 'df_imu'
'data.hamster2__velocity.csv' loaded to 'df_velocity'
'data.tf.csv' loaded to 'df_tf'

Defined Variables:
    df_command
    df_imu
    df_velocity
    df_tf
    df_merge    # merged from 'df_command', 'df_velocity', 'df_imu' and 'df_tf'

Finish!


Loading BokehJS ...

In [2]:
p = plot_config(width=900, height=600, title='0000000',  x_label='t [ s ]', y_label= 'unknown ')

# # linear acceleration from imu
p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_x"].to_numpy(),legend_label='a_x', size = 2, fill_color='red', line_color='red')
p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_y"].to_numpy(),legend_label='a_y', size = 2, fill_color='green', line_color='green')
p.circle(x=df_imu["imu.header.stamp"].to_numpy(), y=df_imu["imu.a_z"].to_numpy(),legend_label='a_z', size = 2, fill_color='blue', line_color='blue')

# p.circle(x=df_imu["imu.header.stamp"].to_numpy()[1900:-1], y=df_imu["imu.a_z"].to_numpy()[1900:-1],legend_label='a_z_', size = 2, fill_color='black', line_color='black')
p.circle(x=df_imu["imu.header.stamp"].to_numpy()[500:1600], y=df_imu["imu.a_z"].to_numpy()[500:1600],legend_label='a_z_', size = 2, fill_color='black', line_color='black')


show(p, notebook_handle=True)



In [2]:
# df_imu["imu.a_x"].to_numpy()[1900:-1]
gx = np.average(df_imu["imu.a_x"].to_numpy()[1900:-1])
gy = np.average(df_imu["imu.a_y"].to_numpy()[1900:-1])
gz = np.average(df_imu["imu.a_z"].to_numpy()[1900:-1])
g = np.array([gx,gy,gz])
norm_g = np.linalg.norm(g)
print("g = ", g)
print("|g| = norm_g =", norm_g)
print()

sample_x = df_imu["imu.a_x"].to_numpy()[500:1600]
sample_y = df_imu["imu.a_y"].to_numpy()[500:1600]
sample_z = df_imu["imu.a_z"].to_numpy()[500:1600]

# find a plane that fits the sample points and its normal
points = np.column_stack((sample_x, sample_y, sample_z))
pca = PCA(n_components=3)
pca.fit(points)
normal = -pca.components_[2]
print("normal: ",normal,'\n')


# e_y
e_y = - g/norm_g

# e_z
z_direc = np.cross(g,normal)# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# z_direc = np.cross(g,np.array([1,0,0]))# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
e_z = z_direc/np.linalg.norm(z_direc)

# e_x
x_direc = np.cross(e_y,e_z)
e_x = x_direc/np.linalg.norm(x_direc)

# Rotation matrix
e_x = e_x.reshape(-1,1)
e_y = e_y.reshape(-1,1)
e_z = e_z.reshape(-1,1)

print('e_x:\n', e_x)
print('e_y:\n', e_y)
print('e_z:\n', e_z)
print()

e_R_o = np.concatenate((e_x,e_y,e_z),axis=1)
o_R_e = np.linalg.inv(e_R_o)
print("e_R_o:")
print(e_R_o)
print("o_R_e:")
print(o_R_e)

# Save o_R_e and e_R_o as CSV file
# pd.DataFrame(o_R_e).to_csv("o_R_e.csv", header=None, index=None)
# pd.DataFrame(e_R_o).to_csv("e_R_o.csv", header=None, index=None)

# %matplotlib notebook
# # Create a 3D figure
# fig = plt.figure()
# ax = plt.axes(projection="3d")
# # Set the x, y, and z limits
# ax.set_xlim([-1, 1])
# ax.set_ylim([-1, 1])
# ax.set_zlim([-1, 1])
# # Plot the data points
# ax.scatter3D(0, 0, 0,s=20,c="k")
# ax.scatter3D(e_x[0][0], e_x[1][0], e_x[2][0],s=20,c="r")
# ax.scatter3D(e_y[0][0], e_y[1][0], e_y[2][0],s=20,c="b")
# ax.scatter3D(e_z[0][0], e_z[1][0], e_z[2][0],s=20,c="g")
# # Set the axis labels and title
# ax.set_xlabel('X-axis')
# ax.set_ylabel('Y-axis')
# ax.set_zlabel('Z-axis')
# ax.set_title('3D Point Diagram')
# # Show the plot
# plt.show()

g =  [ 0.08030556 -9.72724272  0.60994637]
|g| = norm_g = 9.746678127154874

normal:  [ 0.99919678 -0.03435652  0.02062575] 

e_x:
 [[0.99979579]
 [0.00937655]
 [0.01790123]]
e_y:
 [[-0.00823928]
 [ 0.99800595]
 [-0.06257992]]
e_z:
 [[-0.01845231]
 [ 0.06241965]
 [ 0.9978794 ]]

e_R_o:
[[ 0.99979579 -0.00823928 -0.01845231]
 [ 0.00937655  0.99800595  0.06241965]
 [ 0.01790123 -0.06257992  0.9978794 ]]
o_R_e:
[[ 0.99979579  0.00937655  0.01790123]
 [-0.00823928  0.99800595 -0.06257992]
 [-0.01845231  0.06241965  0.9978794 ]]


In [15]:
a_xyz = df_imu[["imu.a_x", "imu.a_y", "imu.a_z"]].to_numpy().T
w_xyz = df_imu[["imu.w_x", "imu.w_y", "imu.w_z"]].to_numpy().T
a_xyz_corrected = o_R_e @ a_xyz
w_xyz_corrected = o_R_e @ w_xyz

p = plot_config(width=900, height=600, title='corrected data',  x_label='t [ s ]', y_label= 'unknown ')
# # linear acceleration from imu
p.circle(x=df_imu["imu.header.stamp"], y=a_xyz_corrected[0],legend_label='a_x', size = 2, fill_color='red', line_color='red')
p.circle(x=df_imu["imu.header.stamp"], y=a_xyz_corrected[1],legend_label='a_y', size = 2, fill_color='green', line_color='green')
p.circle(x=df_imu["imu.header.stamp"], y=a_xyz_corrected[2],legend_label='a_z', size = 2, fill_color='blue', line_color='blue')

# p.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.a_x"],legend_label='a_x_original', size = 1, fill_color='grey', line_color='grey')
# p.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.a_y"],legend_label='a_y_original', size = 1, fill_color='grey', line_color='grey')
# p.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.a_z"],legend_label='a_z_original', size = 1, fill_color='grey', line_color='grey')

p1 = plot_config(width=900, height=600, title='corrected data',  x_label='t [ s ]', y_label= 'unknown ')
# # linear acceleration from imu
p1.circle(x=df_imu["imu.header.stamp"], y=w_xyz_corrected[0],legend_label='w_x', size = 2, fill_color='red', line_color='red')
p1.circle(x=df_imu["imu.header.stamp"], y=w_xyz_corrected[1],legend_label='w_y', size = 2, fill_color='green', line_color='green')
p1.circle(x=df_imu["imu.header.stamp"], y=w_xyz_corrected[2],legend_label='w_z', size = 2, fill_color='blue', line_color='blue')

# p1.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.w_x"],legend_label='w_x_original', size = 1, fill_color='grey', line_color='grey')
# p1.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.w_y"],legend_label='w_y_original', size = 1, fill_color='grey', line_color='grey')
# p1.circle(x=df_imu["imu.header.stamp"], y=df_imu["imu.w_z"],legend_label='w_z_original', size = 1, fill_color='grey', line_color='grey')

show(p, notebook_handle=True)
show(p1, notebook_handle=True)